# Simple stroke system model

![](./images/system_1.png)

## Conversion of arithmetic mean/sd to log normal mean/sd required for model

![](./images/conversion.png)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import simpy

In [2]:
# Parameters
admissions_per_year = 1000

prop_hasu_using_asu = 0.7
pro_hasu_using_esd_only = 0.1
prop_asu_using_esd = 0.5

los_hasu_mean = 2.0
los_hasu_cv = 1.0
los_asu_no_esd_mean = 15
los_asu_no_esd_cv = 1.0
los_asu_with_esd_mean = 15
los_asu_with_esd_cv = 1.0
los_esd_mean = 20
los_esd_cv = 1.0

inter_arrival_time = 365.0 / admissions_per_year

In [3]:
# Convert parameters to log normal equivalents

def log_normal_params(mean, cv):
    sd = mean * cv
    sigma = np.sqrt(np.log(1 + ((sd**2)/(mean**2))))
    mu = np.log(mean) - ((sigma**2)/2)

    return mu, sigma

# Convert parameters to log normal equivalents
los_hasu_mu, los_hasu_sigma = \
    log_normal_params(los_hasu_mean, los_hasu_cv)
los_asu_no_esd_mu, los_asu_no_esd_sigma = \
    log_normal_params(los_asu_no_esd_mean, los_asu_no_esd_cv)
los_asu_with_esd_mu, los_asu_with_esd_sigma = \
    log_normal_params(los_asu_with_esd_mean, los_asu_with_esd_cv)
los_esd_mu, los_esd_sigma = log_normal_params(los_esd_mean, los_esd_cv)

## Define model

(See https://github.com/MichaelAllen1966/1804_python_healthcare/blob/master/jupyter_notebooks/0087_bed_occupancy_object_based.ipynb)

In [4]:
class System:
    """
    System class holds:
    1) Dictionary of patients present
    2) List of audit times
    3) List of beds occupied at each audit time
    4) Current total beds occupied
    5) Admissions to data

    Methods:

    __init__: Set up system instance

    audit: records number of beds occupied

    build_audit_report: builds audit report at end of run (calculate 5th, 50th
    and 95th percentile bed occupancy.

    chart: plot beds occupied over time (at end of run)
    """

    def __init__(self):
        self.patients = {}
        self.audit_time = []
        self.audit_times = []
        self.hasu_beds_occupied = []
        self.asu_beds_occupied = []
        self.esd_beds_occupied = []
        self.admissions = 0
        self.asu_count = 0
        self.hasu_count = 0
        self.esd_count = 0

    def audit(self, time):
        """
        Audit method. When called appends current simulation time to audit_time
        list, and appends current bed counts to lists.
        """
        
        self.audit_time.append(time)
        self.hasu_beds_occupied.append(self.hasu_count)
        self.asu_beds_occupied.append(self.asu_count)
        self.esd_beds_occupied.append(self.esd_count)


    def build_audit_report(self):
        """
        This method is called at end of run. It creates a pandas DataFrame,
        transfers audit times and bed counts to the DataFrame, and 
        calculates/stores 5th, 50th and 95th percentiles.
        """
        self.audit_report = pd.DataFrame()
        self.audit_report['Time'] = self.audit_time
        
        # Record bed counts at each audit time
        self.audit_report['Occupied_hasu_beds'] = self.hasu_beds_occupied
        self.audit_report['Occupied_asu_beds'] = self.asu_beds_occupied
        self.audit_report['Occupied_esd_beds'] = self.esd_beds_occupied

        # Count 


        self.audit_report_summary = pd.Series()
        self.audit_report_summary['hasu_5th_percentile'] = self.audit_report['Occupied_hasu_beds'].quantile(0.05)
        self.audit_report_summary['hasu_median'] = self.audit_report['Occupied_hasu_beds'].quantile(0.5)
        self.audit_report_summary['hasu_95th_percentile'] = self.audit_report['Occupied_hasu_beds'].quantile(0.95)
        self.audit_report_summary['asu_5th_percentile'] = self.audit_report['Occupied_asu_beds'].quantile(0.05)
        self.audit_report_summary['asu_median'] = self.audit_report['Occupied_asu_beds'].quantile(0.5)
        self.audit_report_summary['asu_95th_percentile'] = self.audit_report['Occupied_asu_beds'].quantile(0.95)
        self.audit_report_summary['esd_5th_percentile'] = self.audit_report['Occupied_esd_beds'].quantile(0.05)
        self.audit_report_summary['esd_median'] = self.audit_report['Occupied_esd_beds'].quantile(0.5)
        self.audit_report_summary['esd_95th_percentile'] = self.audit_report['Occupied_esd_beds'].quantile(0.95)
        self.audit_report_summary = self.audit_report_summary.round(1)

    def chart(self):
        """
        This method is called at end of run. It plots beds occupancy over the
        model run, with 5%, 50% and 95% percentiles.
        """

        fig = plt.figure(figsize=(10, 6))
        ax = fig.add_subplot(111)
        x = self.audit_report['Day']
        y1 = self.audit_report['Occupied_hasu_beds']
        y2 = self.audit_report['Occupied_asu_beds']
        y3 = self.audit_report['Occupied_esd_beds']
        ax.plot(x, y1, color='blue', label='hasu')
        ax.plot(x, y2, color='green', label='asu')
        #ax.plot(x, y3, color='red', label='esd')

        ax.fill_between(x, self.audit_report['Occupied_hasu_beds'].quantile(0.05),
                        self.audit_report['Occupied_hasu_beds'].quantile(0.95),
                        color='blue', alpha=0.2)
        ax.fill_between(x, self.audit_report['Occupied_asu_beds'].quantile(0.05),
                        self.audit_report['Occupied_asu_beds'].quantile(0.95),
                        color='green', alpha=0.2)
        #ax.fill_between(x, self.audit_report['Occupied_esd_beds'].quantile(0.05),
        #                self.audit_report['Occupied_esd_beds'].quantile(0.95),
        #                color='red', alpha=0.2)
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Occupancy (people)')
        txt = "Shaded areas show 5th to 9th percentile occupancy"
        ax.text(0.5, 0.95, txt, ha='center', va='center', transform=ax.transAxes,
            bbox=dict(facecolor='white', alpha=0.5))
        ax.legend()
        # Save
        plt.savefig('bed_occupancy.png', dpi=300, bbox_inches='tight')
        # Show
        plt.show()

        
        return
    

In [5]:
class Patient:
    """
    Patient class. 
    The only method is __init__ for creating a patient (with assignment of
    patient id and length of stay).
    """

    def __init__(self, 
        patient_id, hasu_los, asu_los, esd_los, use_asu, use_esd):
        """
        Contructor for new patient.
        """
        self.id = patient_id
        self.los_hasu = hasu_los
        self.los_asu = asu_los
        self.los_esd = esd_los
        self.use_asu = use_asu
        self.use_esd = use_esd

        return

In [6]:
class Model:
    """
    The main model class.

    The model class contains the model environment. The modelling environment is
    set up, and patient arrival and audit processes initiated. Patient arrival
    triggers a spell for that patient in hospital. Arrivals and audit continue
    for the duration of the model run. The audit is then summarised and bed
    occupancy (with 5th, 50th and 95th percentiles) plotted.
    """

    def __init__(self):

        """
        Constructor class for new model.
        """
        self.env = simpy.Environment()

        
    def audit_beds(self, delay):
        """
        Bed audit process. Begins by applying delay, then calls for audit at
        intervals.

        :param delay: delay (days) at start of model run for model warm-up.
        """

        # Delay first audit
        yield self.env.timeout(delay)

        # Continually generate audit requests until end of model run
        while True:
            # Call audit
            self.system.audit(self.env.now)
            # Delay 1 day until next call
            yield self.env.timeout(1)

    def new_admission(self, interarrival_time):
        """
        New admissions to hospital.

        :param interarrival_time: average time (days) between arrivals
        :param los: average length of stay (days)
        """
        while True:
            # Increment hospital admissions count
            self.system.admissions += 1

            # Set patient use of ASU and ESD
            use_asu=random.random() < prop_hasu_using_asu
            if use_asu:
                use_esd_after_asu = random.random() < prop_asu_using_esd
                use_esd_after_hasu = 0
            else:
                use_esd_after_hasu = random.random() < pro_hasu_using_esd_only
                use_esd_after_asu = 0

            # Check any use of ESD
            use_esd = use_esd_after_asu or use_esd_after_hasu
            
            # Set HASU length of stay
            hasu_los = random.lognormvariate(los_hasu_mu, los_hasu_sigma)
            
            # Set ASU length of stay
            if use_asu:
                if use_esd:
                    asu_los = random.lognormvariate(los_asu_with_esd_mu, los_asu_with_esd_sigma)
                else:
                    asu_los = random.lognormvariate(los_asu_no_esd_mu, los_asu_no_esd_sigma)
            else:
                asu_los=0,
            
            # Set ESD length of stay (currently use the same for ESD after HASU or ASU)
            if use_esd_after_asu:
                esd_los = random.lognormvariate(los_esd_mu, los_esd_sigma)
            elif use_esd_after_hasu:
                esd_los = random.lognormvariate(los_esd_mu, los_esd_sigma)
            else:
                esd_los=0

            # Generate new patient
            p = Patient(patient_id=self.system.admissions,
                        hasu_los=hasu_los,
                        asu_los=asu_los,
                        esd_los=esd_los,
                        use_asu=use_asu,
                        use_esd=use_esd                        
            )

            # Add patient to hospital patient dictionary
            self.system.patients[p.id] = p

            # Generate a patient spell in hospital (by calling spell method).
            # This triggers a patient admission and allows the next arrival to
            # be set before the paitent spell is finished
            spell = self.spell(p)
            self.env.process(spell)

            # Set and call delay before looping back to new patient admission
            next_admission = random.expovariate(1 / interarrival_time)
            yield self.env.timeout(next_admission)
            

    def run(self, warm_up, sim_duration):
        """
        Controls the main model run. Initialises model and patient arrival and
        audit processes. Instigates the run. At end of run calls for an audit
        summary and bed occupancy plot
        """

        # Set up hospital (calling Hospital class)
        self.system = System()

        # Set up starting processes: new admissions and bed  audit (with delay)
        self.env.process(self.new_admission(inter_arrival_time))
        self.env.process(self.audit_beds(delay=warm_up))

        # Start model run
        self.env.run(until=sim_duration + warm_up)

        # At end of run call for bed audit summary and bed occupancy plot
        self.system.build_audit_report()
        self.system.chart()

        
    def spell(self, p):
        """Patient spell in HASU/ASU/ESD"""

        # HASU
        self.system.hasu_count += 1
        # Wait for HASU length of stay
        yield self.env.timeout(p.los_hasu)
        self.system.hasu_count -= 1

        # ASU
        if p.use_asu:
            self.system.asu_count += 1
            # Wait for ASU length of stay
            yield self.env.timeout(p.los_asu)
            self.system.asu_count -= 1

        # ESD
        if p.use_esd:
            self.system.esd_count += 1
            # Wait for ESD length of stay
            yield self.env.timeout(p.los_esd)
            self.system.esd_count -= 1

        # Delete patient from system dictionary
        del self.system.patients[p.id]

In [ ]:
warm_up = 100
sim_duration = 365 # Duration after warm up
m = Model()
m.run(warm_up, sim_duration)

In [ ]:
m.system.audit_report_summary